### This part is aleady extracted to .py file, skip it from here

In [ ]:
import os
from glob import glob
import imageio
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from utils import segment_imgs, read_to_rgb, quantify_fov
from covid_screening_quantify import sort_wells_fovs, quantify_all_fovs, combine_all_quantify

In [ ]:
CHS_MAPPING = {
    'nuclei': 'ch1',
    'er': 'ch2',
    'virus': 'ch3'
}

In [ ]:
def sort_wells_fovs(image_folder):
    """
    sort the wells and fovs in an image folder
    
    Args:
        image_folder (str): path of image folder, which hosts all the images.
        
    Returns:
        tuple: wells and fovs list
    """
    files = os.listdir(image_folder)
    files = list(
        filter(
            lambda item:
                item if item.endswith('.tiff') and not item.startswith('.') else None,
            files
        )
    )
    fovs = [file[:9] for file in files]
    fovs = list(set(fovs))
    fovs.sort()
    wells = [fov[:6] for fov in fovs]
    wells = list(set(wells))
    wells.sort()
    
    return wells, fovs, files

In [ ]:
def quantify_all_fovs(image_folder, processed_folder=None, save_mask=True, quantify_directly=True):
    """
    Quantify all the fovs in image folder.
    
    Args:
        image_folder (str): path to folder where hosts all the images.
        save_mask (bool): to save mask as file or not.
        quantify_directly (bool): to quantify the cell data directly or not.
    
    Returns:
        str: the path of processed folder.
    """
    wells, fovs, files = sort_wells_fovs(image_folder)
    # create processed folder if to save files
    if not processed_folder:
        processed_folder = image_folder + '_processed'
    if save_mask or quantify_directly:
        os.makedirs(processed_folder, exist_ok=True)

    for _, well_id in enumerate(wells):
        # get all the fovs for the well
        well_fovs = list(
            filter(
                lambda fov: fov if fov[:6] == well_id else None, fovs
            )
        )
        well_fovs.sort()
        for _, well_fov in enumerate(well_fovs):
            # set the output for fov_data_output
            fov_data_output = []
            # get all the files for a fov
            fov_files = list(
                filter(
                    lambda file: file if well_fov in file else None, files
                )
            )
            fov_files.sort()
            # this part is a simple assertation to make sure only one plane in zstack.
            # Otherwise, changes are needed fot this pipeline
            planes = [fov_file.split('-')[0] for fov_file in fov_files]
            planes = set(planes)
            assert len(planes) == 1, "This pipeline requires single plane image, but multiple planes/z-stack images found!"

            #CHS_MAPPING
            nuclei_file = list(
                filter(
                    lambda fov_file:
                        fov_file if fov_file.split('-')[1][:3] == CHS_MAPPING['nuclei'] else None,
                    fov_files
                )
            )
            er_file = list(
                filter(
                    lambda fov_file:
                        fov_file if fov_file.split('-')[1][:3] == CHS_MAPPING['er'] else None,
                    fov_files
                )
            )
            virus_file = list(
                filter(
                    lambda fov_file:
                        fov_file if fov_file.split('-')[1][:3] == CHS_MAPPING['virus'] else None,
                    fov_files
                )
            )
            assert len(nuclei_file) == len(er_file) == len(virus_file) == 1, "{} should have only one file for each channel!".format(well_fov)

            fov_chs_files = {
                'nuclei': os.path.join(image_folder, nuclei_file[0]),
                'er': os.path.join(image_folder, er_file[0]),
                'virus': os.path.join(image_folder, virus_file[0])
            }

            # read image files to rgb img array with virus in r, er in g, nuclei in b
            fov_chs_rgb, channel =  read_to_rgb(fov_chs_files)

            # segment the cell images
            results = segment_imgs([fov_chs_rgb], [channel])
            mask = results['masks'][0]
            # save mask data to file, if save_mask is true
            if save_mask:
                mask = np.asarray(mask, dtype='uint16')
                mask_file = os.path.join(processed_folder, well_fov + '_mask.png')
                imageio.imwrite(mask_file, mask, format='PNG-FI')

            # skip the quantificaiton if quantify_directly is false
            if not quantify_directly:
                continue
            # calculate the intensity
            fov_quantify = quantify_fov(fov_chs_rgb[:,:,0], mask)
            # if no cells, continue
            if not len(fov_quantify):
                continue

            # write data to list, and then to csv file
            for cell_data in fov_quantify:
                cell_idx, cell_size, cell_integ, cell_mean = cell_data

                cell_info = {}
                cell_info['well_id'] = well_id
                cell_info['well_fovs'] = well_fov
                cell_info['nuclei_file'] = fov_chs_files['nuclei']
                cell_info['er_file'] = fov_chs_files['er']
                cell_info['virus_file'] = fov_chs_files['virus']
                cell_info['cell_idx'] = cell_idx
                cell_info['cell_size'] = cell_size
                cell_info['cell_integ'] = cell_integ
                cell_info['cell_mean'] = cell_mean
                # append the data to fov_data_output
                fov_data_output.append(cell_info)
            # save the data from each fov to csv file
            df = pd.DataFrame(fov_data_output)
            csv_file = os.path.join(processed_folder, well_fov + '_quantify.csv')

            df.to_csv(csv_file, index=False)
    
    return processed_folder

In [ ]:
def combine_all_quantify(processed_folder, combine_file=True):
    """
    Combine fov quantify csv into one combined csv file.
    
    Args:
        processed_folder (str): the path of processed folder.
        combine_file (bool): save combined csv if true, else not.
    """
    quantify_csvs = glob(processed_folder + '/*_quantify.csv')
    quantiy_csvs = list(
        filter(
            lambda quantify_csv:
                quantify_csv if not os.path.basename(quantify_csv).startswith('.') else None,
            quantify_csvs
        )
    )
    quantiy_csvs.sort()
    dfs = []
    for _, quantify_csv in enumerate(quantify_csvs):
        try:
            df = pd.read_csv(quantify_csv)
            dfs.append(df)
        except Exception as e:
            print(f'{quantify_csv} cannot be successfully read!')
    df = pd.concat(dfs)
    if combine_file:
        df.to_csv(
            '/home/haoxu/data/test_data_20201218/40x_processed/quantify_all.csv',
            index=False
        )
    else:
        return df

### skip till here

### do test from here

In [1]:
from covid_screening_quantify import sort_wells_fovs, quantify_all_fovs, combine_all_quantify

** TORCH CUDA version installed and working. **


In [2]:
# test parameter
image_folder = '/home/haoxu/data/test_data_20201218/40x'

In [3]:
quantify_all_fovs(image_folder)

** TORCH CUDA version installed and working. **
>>>> using GPU
processing 1 image(s)
computing styles from images
time spent: running network 0.15s; flow+mask computation 1.59
estimated cell diameters for 1 image(s) in 2.17 sec
>>> diameter(s) =  [72.92151461]


/home/haoxu/miniconda3/envs/cellpose_torch/lib/python3.7/site-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


time spent: running network 0.37s; flow+mask computation 0.23
estimated masks for 1 image(s) in 0.64 sec
>>>> TOTAL TIME 2.81 sec
** TORCH CUDA version installed and working. **
>>>> using GPU
processing 1 image(s)
computing styles from images
time spent: running network 0.21s; flow+mask computation 2.12
estimated cell diameters for 1 image(s) in 2.75 sec
>>> diameter(s) =  [79.18374949]
time spent: running network 0.38s; flow+mask computation 0.19
estimated masks for 1 image(s) in 0.61 sec
>>>> TOTAL TIME 3.36 sec
** TORCH CUDA version installed and working. **
>>>> using GPU
processing 1 image(s)
computing styles from images
time spent: running network 0.14s; flow+mask computation 1.95
estimated cell diameters for 1 image(s) in 2.52 sec
>>> diameter(s) =  [94.46133999]
time spent: running network 0.26s; flow+mask computation 0.17
estimated masks for 1 image(s) in 0.47 sec
>>>> TOTAL TIME 3.00 sec
** TORCH CUDA version installed and working. **
>>>> using GPU
processing 1 image(s)
com

'/home/haoxu/data/test_data_20201218/40x_processed'

In [ ]:
#image_folder = os.path.join(acquisition_folder, "Images")
#image_folder = '/home/haoxu/data/test_data_20201218/40x'
#save_mask = False
#quantify_directly = True
#process_folder = image_folder + '_processed'
#if save_mask or quantify_directly:
#    os.makedirs(process_folder, exist_ok=True)

In [ ]:
# set up function
    # image reader
    # segmentation
    # quantification
    # put all quantification into one file
    # define cell infection
    # verify cell_infectionS
    # statistics output

In [4]:
processed_folder = '/home/haoxu/data/test_data_20201218/40x_processed'

In [5]:
combine_all_quantify(processed_folder)

In [ ]:
quantiy_csvs

In [ ]:
df = pd.read_csv('/home/haoxu/data/test_data_20201218/40x_processed/quantify_all.csv')

In [ ]:
df

In [ ]:
intensity = df.cell_mean.to_list()

In [ ]:
plt.figure(figsize=(10, 10))
plt.hist(intensity, bins=70, range=(min(intensity), max(intensity)))